In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)

In [2]:
mnist = input_data.read_data_sets("../data/MNIST_data/", one_hot=True)

Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# hyper parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 100

In [4]:
# the model to build
class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
            # set training mode for dropout
            self.is_training = tf.placeholder(tf.bool, name='is_trainig_mode')
            
            # input placeholders
            self.X = tf.placeholder(tf.float32, [None, 784], name='X')
            self.Y = tf.placeholder(tf.float32, [None, 10], name='Y')
            X_img = tf.reshape(self.X, [-1, 28, 28, 1], name='X_img')
            
            # layer1: conv
            L1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)
            L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2, 2], padding='SAME', strides=2)
            L1 = tf.layers.dropout(inputs=L1, rate=0.7, training=self.is_training)
            
            # layer2: conv
            L2 = tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)
            L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2, 2], padding='SAME', strides=2)
            L2 = tf.layers.dropout(inputs=L2, rate=0.7, training=self.is_training)
            
            # layer3: conv
            L3 = tf.layers.conv2d(inputs=L2, filters=128, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)
            L3 = tf.layers.max_pooling2d(inputs=L3, pool_size=[2, 2], padding='SAME', strides=2)
            L3 = tf.layers.dropout(inputs=L3, rate=0.7, training=self.is_training)
            
            # layer4: fully connected
            L3_flat = tf.reshape(L3, [-1, 128 * 4 * 4])
            L4 = tf.layers.dense(inputs=L3_flat, units=625, activation=tf.nn.relu)
            L4 = tf.layers.dropout(inputs=L4, rate=0.5, training=self.is_training)
            
            # final layer
            self.logits = tf.layers.dense(inputs=L4, units=10)
            
        # cost & optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
        
        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    def predict(self, x_test, training=False):
        return self.sess.run(self.logits, feed_dict={self.X: x_test, self.is_training: training})
    
    def get_accuracy(self, x_test, y_test, training=True):
        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y: y_test, self.is_training: training})
    
    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: x_data, self.Y: y_data, self.is_training: training})


In [5]:
# initialize session
sess = tf.Session()

In [6]:
# create multiple models
models = []
num_models = 10  # 2: 0.9914, 10: 0.9917
for m in range(num_models):
    models.append(Model(sess, 'model_' + str(m)))

In [7]:
# initialize variables
sess.run(tf.global_variables_initializer())

In [8]:
print('Learning Started!')

# train my model
for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        # train each model
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

print('Learning Finished!')

Learning Started!
Epoch: 0001 cost = [ 0.80186316  0.77568675  0.74748487  0.79288081  0.77858696  0.79969638
  0.74948728  0.75341648  0.7896797   0.78207075]
Epoch: 0002 cost = [ 0.29005169  0.29451718  0.28792719  0.29716553  0.29229922  0.2920417
  0.29104502  0.29045737  0.29815696  0.29127063]
Epoch: 0003 cost = [ 0.22920355  0.2306721   0.22731929  0.23127657  0.2322948   0.23056679
  0.23324648  0.23079267  0.23595522  0.22615084]
Epoch: 0004 cost = [ 0.19608979  0.20363097  0.19841447  0.1987681   0.20396192  0.19982144
  0.20148488  0.20222247  0.2062789   0.19780503]
Epoch: 0005 cost = [ 0.18355306  0.18477394  0.18503631  0.18527276  0.18809424  0.1813539
  0.18269716  0.18700593  0.18418929  0.18239645]
Epoch: 0006 cost = [ 0.16616918  0.17209198  0.16483162  0.16927533  0.17742989  0.17249053
  0.174654    0.17418641  0.17814687  0.16770007]
Epoch: 0007 cost = [ 0.16406126  0.164169    0.16211505  0.15931919  0.16454028  0.16280193
  0.15754638  0.16496893  0.16887667  0.

In [9]:
# Test model and check accuracy
test_size = len(mnist.test.labels)
predictions = np.zeros(test_size * 10).reshape(test_size, 10)
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(
        mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    
    # sum all predictions from each model(prediction is softmax output)
    predictions += p

0 Accuracy: 0.9617
1 Accuracy: 0.9563
2 Accuracy: 0.9615
3 Accuracy: 0.9652
4 Accuracy: 0.9648
5 Accuracy: 0.9641
6 Accuracy: 0.9615
7 Accuracy: 0.9617
8 Accuracy: 0.9645
9 Accuracy: 0.9621


In [10]:
ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))

Ensemble accuracy: 0.9917
